<a href="https://colab.research.google.com/github/Cipe96/EEG-Recognition/blob/main/Classificatore_Ensamble.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<font size=6>**EEG Recognition: Classificatore Ensamble**</font>
</br><font size=3>*Marco Cipollina, Riccardo Era*</font>


<p style="font-size:4px;" align="justify">Dopo avere addestrato il convolutivo costruito per questo progetto sia sui dati della run 01 (basale, basale occhi aperti), sia run 02 (basale, occhi chiusi) che sui dati della run 06 (immaginare di chiedere pugni e piedi in determinati momenti) , in questo notebook vogliamo analizzare le capacità di un classificatore Ensamble (formato dalla media della somma pesata per classe dei tre classificatori) di classificare i dati di una qualsiasi altra run.</p>

<p style="font-size:4px;" align="justify">....</p>

In [1]:
from sklearn.metrics import accuracy_score
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
from google.colab import drive
import tensorflow as tf
import pandas as pd
import numpy as np
import sys
import os

SEED = 96 #Impostiamo un seme specifico per garantire la replicabilità degli esperimenti

In [2]:
%%capture
drive.mount('/content/drive', force_remount=True)
# elimina la cartella sample_data creata automaticamente
! rm -r /content/sample_data

In [3]:
#@title Percorso della cartella del progetto su Google Drive:

#@markdown Se la cartella del progetto si trova nella root di Drive, scrivere solo il suo nome:
PERCORSO_DRIVE = "EEG Recognition" #@param {type:"string"}

PERCORSO_DRIVE = '/content/drive/MyDrive/' + PERCORSO_DRIVE

In [4]:
sys.path.append(PERCORSO_DRIVE)                       # ci permetterà di importare le funzioni presenti in altri file
from shared_utilities import convolutivo              # importare il classificatore convolutivo

<a name="2"></a>
# **Download**

In [5]:
#@title Seleziona la Run che vuoi utilizzare (fra quelle che hai sul drive):
#@markdown Inserisci un valore compreso fra 1 e 14 inclusi:

run = 5 #@param {type:"integer"}
run = f"R{run:02d}"


In [6]:
#@title Seleziona la banda che vuoi utilizzare:
#@markdown (delta, theta, alpha, beta, gamma, broadband, personalizzata, ABG)

banda = 'ABG' #@param ['delta', 'theta', 'alpha', 'beta', 'gamma', 'broadband', 'personalizzata', 'ABG']
save_path = f'{PERCORSO_DRIVE}/Dati preprocessati/{run}/{banda}'

In [7]:
# Carica i dati e le etichette del test set
test_data = np.load(f"{save_path}/test_data_{banda}.npy")
test_labels = np.load(f"{save_path}/test_labels_{banda}.npy")

Controlliamo che la shape sia coerente con quella che ci servirà per il modello di classificazione

In [8]:
# Verifica le dimensioni dei dataset e delle etichette
print(f"Test set shape: {test_data.shape}, Test labels shape: {test_labels.shape}")

Test set shape: (1409, 240, 64), Test labels shape: (1409,)


Effettuiamo uno shuffle del dataset per evitare di dare in pasto i dati al modello in maniera ordinata, permettendo così un addestramento meno condizionato.

In [10]:
#shuffle dei dataset e ettichette con stesso ordine di shuffle
test_data, test_labels = shuffle(test_data, test_labels, random_state=SEED)


<a name="3"></a>
# **Convolutivo addestrato su Run 01**

In [11]:
#@title Carica il modello (architettura, pesi e configurazioni)
weights_path = f'{PERCORSO_DRIVE}/Dati preprocessati/R01/Pesi convolutivo/'
model01 = tf.keras.models.load_model(weights_path + f'best-{banda}.keras')
print("architettura, pesi e configurazione caricati con successo")


architettura, pesi e configurazione caricati con successo


Verifichiamo che il modello sia costruito come vogliamo.

In [12]:
model01.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 240, 64)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ separable_conv1d_1 (SeparableConv1D) │ (None, 240, 64)             │           4,352 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_2                │ (None, 240, 64)             │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 240, 64)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling1d             │ (None, 64)                  │               0 │
│ (GlobalAveragePooling1D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 109)                 │           7,085 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 34,825 (136.04 KB)

 Trainable params: 11,565 (45.18 KB)

 Non-trainable params: 128 (512.00 B)

 Optimizer params: 23,132 (90.36 KB)

Preleviamo le probabilità per ogni campione dal modello addestrato su Run 01

In [13]:
predizioni01 = model01.predict(test_data)

45/45 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step


<a name="3"></a>
# **Convolutivo addestrato su Run 02**

In [14]:
#@title Carica il modello (architettura, pesi e configurazioni)
weights_path = f'{PERCORSO_DRIVE}/Dati preprocessati/R02/Pesi convolutivo/'
model02 = tf.keras.models.load_model(weights_path + f'best-{banda}.keras')
print("architettura, pesi e configurazione caricati con successo")


architettura, pesi e configurazione caricati con successo


Verifichiamo che il modello sia costruito come vogliamo.

In [15]:
model02.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 240, 64)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ separable_conv1d_1 (SeparableConv1D) │ (None, 240, 64)             │           4,352 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_2                │ (None, 240, 64)             │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 240, 64)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling1d             │ (None, 64)                  │               0 │
│ (GlobalAveragePooling1D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 109)                 │           7,085 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 34,825 (136.04 KB)

 Trainable params: 11,565 (45.18 KB)

 Non-trainable params: 128 (512.00 B)

 Optimizer params: 23,132 (90.36 KB)

Preleviamo le probabilità per ogni campione dal modello addestrato su Run 01

In [16]:
predizioni02 = model02.predict(test_data)

45/45 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step


<a name="3"></a>
# **Convolutivo addestrato su Run 06**

In [17]:
#@title Carica il modello (architettura, pesi e configurazioni)
weights_path = f'{PERCORSO_DRIVE}/Dati preprocessati/R06/Pesi convolutivo/'
model06 = tf.keras.models.load_model(weights_path + f'best-{banda}.keras')
print("architettura, pesi e configurazione caricati con successo")


architettura, pesi e configurazione caricati con successo


Verifichiamo che il modello sia costruito come vogliamo.

In [18]:
model06.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 240, 64)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ separable_conv1d_1 (SeparableConv1D) │ (None, 240, 64)             │           4,352 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_2                │ (None, 240, 64)             │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 240, 64)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling1d             │ (None, 64)                  │               0 │
│ (GlobalAveragePooling1D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 109)                 │           7,085 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 34,825 (136.04 KB)

 Trainable params: 11,565 (45.18 KB)

 Non-trainable params: 128 (512.00 B)

 Optimizer params: 23,132 (90.36 KB)

Preleviamo le probabilità per ogni campione dal modello addestrato su Run 06

In [19]:
predizioni06 = model06.predict(test_data)

45/45 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step


<a name="3"></a>
# **Classificatore Ensamble**

In [36]:
#@title Combina i voti e calcola la classe più votata
#visto che sulla run 06 abbiamo addestrato col doppio dei dati, diamo il doppio del peso
predizioni = (predizioni01 + predizioni02 + 2*predizioni06) / 4
predizioni_finali = np.argmax(predizioni, axis=1)

In [37]:
#@title Calcola l'accuratezza confrontando le predizioni finali con le etichette
accuratezza = accuracy_score(test_labels, predizioni_finali)
print(f"Accuratezza dell'ensemble: {accuratezza:.2%}")

Accuratezza dell'ensemble: 91.27%
